# A Simplfied Guide to Running Pysrim:

This notebook will serve as a guide to running pysrim as well as parsing important files. Before running pysrim it is first necessary to install SRIM on your computer. If that requirement is already satisfied, then pysrim can simply be installed by running the command "pip install pysrim" in the terminal. Then pysrim can be imported by running the below commands.

In [1]:
#import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from srim import TRIM, SR, Ion, Layer, Target
from srim.output import Results, Collision
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

Note: It may be the case that there is an error when trying to import pysrim for the first time. If this is the case, the problem may be remedied by navagating to this site https://rmlmcfadden.github.io/ion-implantation/pysrim/#caveats, and modifying the source code as mentioned in the section titled "Caveats". Once you modify the code accordingly, it should run correctly. 

Now that everything is properly imported, you're now ready to run pysrim. You start by specifying the kind of ion you want to fire into the target as well as the target medium itself. Below is an example for firing an argon ion of 40 keV into a target of gaseous argon. Note that SRIM always assumes that the target is at STP as mentioned in the textbook. For documentation on the input paramters for Ion, Layer, and every other class in pysrim see: https://pysrim.readthedocs.io/en/latest/genindex.html.

In [2]:
#specify the type of ion you wish to fire into the target
ion= Ion('Ar',energy=40e3 )

In [3]:
#Specify the layers that compose the target medium
layer = Layer({
        'Ar': {
            'stoich': 1.0,
            'E_d': 5.0,
            'lattice': 1.0,
            'surface': 2.0
        }}, density=0.001784, width=100000000.0 ,phase=1)

In [4]:
# construct your target based on the layers you just defined
target = Target([layer])

Now it is time to give the input parameters for TRIM. This is by far the most important part of running the simulation. If you don't do this part correctly, you'll likely have to start the simulation over, as many of paramters inputted here cannot be modified in the TRIM window. The full documentation of the parameters that can be put into the TRIM function can be found at: https://pysrim.readthedocs.io/en/latest/source/srim.html#srim.srim.TRIMSettings. Within the function, you first input the target and ion that you defined above. Next, specify the number of ions you wish to simulate. For the type of calculation you wish to do, read the documentation linked above about which number you need to specify. To learn more about what the different damage calculations are, consult chapter 8 of the SRIM textbook. 

Next, you need to tell TRIM whether it should include the COLLISON.txt and EXYZ.txt files. These are by far the most information files, so make sure to include then. Note that you can only produce the COLLISON.txt files for certain types of damage calculations. Calculation types 2 and 3 should do the trick. To include the COLLISON.txt file, simply set collisions=True. For the EXYZ.txt file you simply need to tell it the energy interval that it should take data at. To learn more about the capabilities of these files, consult chapters 8 and 9 of the SRIM textbook. 

In [5]:
trim = TRIM(target, ion, number_ions=500, calculation=2, collisions= True, exyz=100, ranges= True)

Now its finally time to run the simulation. Once you've specfied everything above, simply specify the path to where SRIM is installed on your computer and then input that into the trim.run function. If everything ran correctly, you should see the TRIM window pop up and start calculating. One final thing to mention, if you wish to obtain the COLLISON.txt type C file, there is only one way that I know of to do this. Immediatly as the TRIM calculation, hit the button at the top that say "Pause TRIM" to pause the calculation. Then towards the bottom left corner of the window, above where it says 3d plots, you should see a little box called "Collision Details". It should already be checked. You need to uncheck then recheck the box. You should get two prompts, one about wanting to keep data about recoils, respond yes to both. TRIM should now produce the type C file. If you dont do any of this, TRIM will produce a type B file.

In [6]:
executable_directory = 'PATH TO SRIM-2013'

In [7]:
results = trim.run(executable_directory)

Once the calculation is finished, you want to be able to create a file where you can store all of the information produced in the simulation, you can do so by running the below commands.

In [10]:
#specify the path to an output directory you want to create to store all the data
output_directory = 'PATH TO OUTPUT DIRECTORY'

In [11]:
#make the directory and then copy the output files there
os.makedirs(output_directory, exist_ok=True)
TRIM.copy_output_files('PATH TO SRIM-2013', output_directory)

For whatever reason, the EXYZ.txt file often doesn't like to transfer over to your newly created file along side everything else. You'll likely to find it in the directory called "SRIM Outputs". You'll likely need to manually transfer this one over to where everything else is stored. If you don't do this by the next time you run a simulation, the file will be overwritten and you'll lose all of the data stored in the EXYZ.txt file from your last simulation :( . So make sure to transfer this over before running the next simulation. Besides that all of your data should be stored in the folder you created, and you should be good for data analysis. 